This notebook is freely available for redistribution under the [GPL-3.0 license](https://choosealicense.com/licenses/gpl-3.0/).

Author: 蘇嘉冠 feabries@gmail.com

Contributors: 蘇郁翔 asdfghjkl225836@gmail.com

# 人工智慧與工程應用實作 2023 作業六

請先登入 Google 帳號並且複製一份至你的 Google Drive，將複製的那份改名為 `hw6_學號.ipynb`，並且用複製的那份寫作業

請記得改變 colab 設定：`編輯` -> `筆記本設定` -> 將 `硬體加速器` 改為 `GPU`

修改成功後執行以下指令，可以看到 GPU、驅動程式（Driver）與 CUDA 等資訊

In [ ]:
!nvidia-smi

## 韓文新聞分類

![](https://www.allkpop.com/upload/2021/12/content/010858/1638367105-untitled.png)

（[圖片來源](https://www.allkpop.com/article/2021/12/korean-and-international-media-outlets-highlight-jimin-in-news-reports-on-bts-ptd-on-stage-la-concert-and-press-conference)）

這次作業的資料修改自 [YNAT](https://klue-benchmark.com/tasks/66/overview/description) 資料集，目的是藉由韓文的新聞標題，來判斷該新聞的主題分類

訓練資料（`train.csv`） 總共有 40,000 筆新聞的資料，包含以下欄位：
- `id`: 該新聞的 ID
- `title`: 該新聞的標題，例如 `G20 때 홍콩 문제 알리자…홍콩 시민들 릴레이 시위 예고`
- `topic`: 該新聞的主題，例如 `사회`

`tppic` 會有 7 種可能：
- `IT과학`: 資訊與科技
- `경제`: 經濟
- `사회`: 社會
- `생활문화`: 生活文化
- `세계`: 國際
- `스포츠`: 運動
- `정치`: 政治

驗證資料（`val`）有 5,000 筆，欄位與訓練資料相同，用來幫助確認你的模型是對的

測試資料（`test.csv`）有 5,000 筆，與訓練資料不同的是缺少 `topic` 欄位

你的目標是要用訓練資料來訓練模型，預測測試資料中的每則新聞的標題（上述 7 種的其中一個，屬於 Multi-Class Classification 任務），並將結果儲存在 `test_pred.csv`，檔案有兩個欄位：
- `id`：該新聞的 ID
- `title`: 該新聞的標題
- `topic`：你的模型預測值

我們會在用真實的答案跟你上傳的檔案（`test_pred.csv`）的 `topic` 欄位的內容計算 Accuracy，並且根據 Accuracy 計算得分（小數點皆無條件進位）：
- 如果 Accuracy >= 0.85，得分為 100
- 如果 Accuracy < 0.85，得分為 *(Accuracy / 0.85) * 100*

例如 Accuracy 為 0.75，分數會是 *(0.75 / 0.85) * 100 = 89*

In [ ]:
!pip install pandas scikit-learn

### 讀取資料

分別讀取訓練、驗證、測試資料，並且簡單秀一下資料的狀況。可以看到訓練與驗證資料擁有完整的欄位，而測試資料少了 `topic` 欄位

In [ ]:
import pandas as pd

train_raw_df = pd.read_csv("https://raw.githubusercontent.com/AINTUT/code_2023/main/datasets/korean_news_topic/train.csv")

print("Number of samples in training data:", len(train_raw_df))
print("Columns of training data:", list(train_raw_df.columns))
print()
print("First 10 samples in training data")
print(train_raw_df.head(10))
print()
print("Topic distribution for training data:")
print(train_raw_df["topic"].value_counts())

In [ ]:
val_raw_df = pd.read_csv("https://raw.githubusercontent.com/AINTUT/code_2023/main/datasets/korean_news_topic/val.csv")

print("Number of samples in validation data:", len(val_raw_df))
print("Columns of validation data:", list(val_raw_df.columns))
print()
print("First 10 samples in validation data")
print(val_raw_df.head(10))
print()
print("Topic distribution for validation data:")
print(val_raw_df["topic"].value_counts())

In [ ]:
test_raw_df = pd.read_csv("https://raw.githubusercontent.com/AINTUT/code_2023/main/datasets/korean_news_topic/test.csv")

print("Number of samples in testing data:", len(test_raw_df))
print("Columns of testing data:", list(test_raw_df.columns))
print()
print("First 10 samples in testing data")
print(test_raw_df.head(10))

### 修改區

接下來你的任務就是試著訓練模型，對測試資料做預測

實做的套件可以考慮以下擇一：
- [Hugging Face Transformers](https://huggingface.co/): 比較困難，但可以有較多客製化，請參考 [課堂練習](https://colab.research.google.com/drive/1ZhA2GBC7BggtlJ6jPpCJXfO2ciod1OWU?usp=sharing)、[官方教學1](https://huggingface.co/docs/transformers/tasks/sequence_classification) 以及 [官方教學2](https://huggingface.co/transformers/v3.2.0/custom_datasets.html)
- [Simple Transformers](https://simpletransformers.ai/): 比較簡單，客製化程度較低，請參考官方網站以及 [這份範例](https://colab.research.google.com/drive/1E2rSeWfOejnSoIuSUwBss9izE80NSZ62?usp=sharing)

幾個建議與提示：
- 用別人預訓練好的 BERT 模型，再去做 Fine-Tuning。可以在 Hugging Face 網站的 Models，搜尋用韓文（或是多國語言）預訓練的模型（[搜尋範例](https://huggingface.co/models?language=ko)）
- 模型架構除了 `bert` 之外，也可以試試其他衍生架構，例如 `roberta`, `roberta`, `xlnet`, `albert`, `electra` 等
- 有些模型架構會有 `base` 或 `large` 的區別，也可以嘗試看看
- 如果想要調整 Hyperparameter，可以參考 [這裡](https://wandb.ai/amogkam/transformers/reports/Hyperparameter-Optimization-for-HuggingFace-Transformers--VmlldzoyMTc2ODI)
- 一開始訓練時，不要全部的訓練資料都丟下去，可以先只拿一小部份（例如 500 筆）去訓練看看，確認方向是對的，再多加一點資料（例如 5,000 筆）看看，都沒問題再全部都下去訓練
- 訓練好模型後，記得用驗證資料算 Accuracy，可以讓你知道模型是否有正確的訓練
- 在這次作業中，驗證資料的 Accuracy 跟測試資料的 Accuracy 應該不會差太多，所以當 Accuracy 覺得差不多時就可以繳交作業惹（當然也很歡迎嘗試做更多改進）


如果你不想訓練模型，別種考量是用 [Zero-Shot Classification](https://huggingface.co/tasks/zero-shot-classification)，或是對（類似）ChatGPT 的設計 prompt 去做 Text Classification

In [ ]:
# Let's do somethig now!

### 關於輸出的檔案格式

你需要的輸出的 `test_pred.csv` 其實就是 `test.csv` 加上 `topic` 欄位

以下範例是隨機亂猜 `topic` 欄位，並且輸出到檔案 `test_pred.csv`。你可以把 `pred_topics` 變數的內容改成你的模型預測結果

In [ ]:
import random

import pandas as pd

test_raw_df = pd.read_csv("https://raw.githubusercontent.com/AINTUT/code_2023/main/datasets/korean_news_topic/test.csv")

target_topics = [
    "IT과학",
    "경제",
    "사회",
    "생활문화",
    "세계",
    "스포츠",
    "정치",
]

pred_topics = [random.choice(target_topics) for _ in range(len(test_raw_df))]
print("\", \"".join(pred_topics[:10]))

test_pred_df = pd.DataFrame({
    "id": test_raw_df["id"],
    "title": test_raw_df["title"],
    "topic": pred_topics,
})
test_pred_df.to_csv("test_pred.csv", index=False)

print(test_pred_df)

假設你已經有 `test_pred.csv`，以下程式碼可以檢查你的檔案格式是否正確，如果正確會輸出 `[PASS] Your file format is correct!`，不正確的話則輸出 `[FAIL] ...` 的訊息

In [ ]:
import pandas as pd

correct_columns = [
    "id",
    "title",
    "topic",
]

target_topics = [
    "IT과학",
    "경제",
    "사회",
    "생활문화",
    "세계",
    "스포츠",
    "정치",
]

def check_format(pred_path):
    test_raw_df = pd.read_csv("https://raw.githubusercontent.com/AINTUT/code_2023/main/datasets/korean_news_topic/test.csv")
    test_pred_df = pd.read_csv(pred_path)

    pred_columns = test_pred_df.columns
    if (len(pred_columns) != len(correct_columns)) or (any(pred_columns != correct_columns)):
        print("[\x1b[31mFAIL\x1b[0m] Columns must be: {}".format(correct_columns))
        return

    if not test_raw_df["id"].equals(test_pred_df["id"]):
        print("[\x1b[31mFAIL\x1b[0m] Row(s) is modified in `id` column")
        return

    if not test_raw_df["title"].equals(test_pred_df["title"]):
        print("[\x1b[31mFAIL\x1b[0m] Row(s) is modified in `title` column")
        return

    if not all(test_pred_df["topic"].isin(target_topics)):
        print("[\x1b[31mFAIL\x1b[0m] Contain non-target value in `topic` column".format(target_topics))
        return

    print("[\033[92mPASS\x1b[0m] Your file format is correct!")

check_format("test_pred.csv")

關於分數計算，我們會用你上傳的 `test_pred.csv` 的 `topic` 欄位與真實的答案計算 Accuracy，具體而言是以 scikit-learn 的 [accuracy_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html) 來做計算

In [ ]:
from sklearn.metrics import accuracy_score

y_gt = ["경제", "생활문화", "경제", "정치", "사회", "경제", "생활문화", "IT과학", "스포츠", "사회"]
y_pred = ["정치", "사회", "정치", "세계", "사회", "경제", "스포츠", "스포츠", "정치", "경제"]
print(accuracy_score(y_gt, y_pred))